In [5]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

In [ ]:
# System parameters
m = 1.0   # mass (kg)
I = 0.01  # moment of inertia (kg*m^2)
g = 9.81  # gravity (m/s^2)
L = 0.2   # arm length (m)

# Optimization parameters
N = 100  # number of control intervals
T = 2.0  # total time (s)
dt = T / N


# define dynamics
nq = 3    # [x, y, phi]
nv = 3    # [x_dot, y_dot, phi_dot]
nu = 2    # [P1, P2]
cq = casadi.SX.sym("q", nq)
cv = casadi.SX.sym("v", nv)
cu = casadi.SX.sym("u", nu) 

# Total thrust
F = cu[0] + cu[1]

# Torque
tau = L * (cu[1] - cu[0])

x_ddot = -F * casadi.sin(cq[2]) / m
y_ddot = F * casadi.cos(cq[2]) / m - g
theta_ddot = tau / I

aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [casadi.vertcat(x_ddot, y_ddot, theta_ddot)])

# Euler integrator
def euler_integrate(q, v, u):
    q_next = q + v * dt
    v_next = v + aba_fn(q, v, u) * dt
    return q_next, v_next

# Set up optimization
opti = casadi.Opti()

Q = opti.variable(nq, N + 1)
V = opti.variable(nv, N + 1)
U = opti.variable(nu, N)

# Set initial state and velocity
initial_state = np.array([-1, 0, 0])
initial_velocity = np.array([0, 0, 0])
opti.subject_to(Q[:, 0] == initial_state)
opti.subject_to(V[:, 0] == initial_velocity)

# Final state
final_state = np.array([1, 0, -4 * np.pi])
final_velocity = np.array([0, 0, 0])
opti.subject_to(Q[:, N] == final_state)
opti.subject_to(V[:, N] == final_velocity)

# Dynamic constraints
for k in range(N):
    q_next, v_next = euler_integrate(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

# Thrust limits
thrust_limit = 20
for k in range(N):
    opti.subject_to(opti.bounded(0.0, U[:, k], thrust_limit))

# Objective function
cost = 0
for k in range(N):
    cost += casadi.sumsqr(U[:, k])  # minimize control effort
    cost += casadi.sumsqr(V[:, k])  # velocities small
opti.minimize(cost)

# Set solver options
p_opts = {"expand": True}
s_opts = {"max_iter": 1000}
opti.solver("ipopt", p_opts, s_opts)

# Solve
sol = opti.solve()

# Extract solution
Q_traj = sol.value(Q)
U_traj = sol.value(U)

# Animation
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(-2, 2)
ax.set_ylim(-1, 3)
ax.set_aspect('equal')
ax.grid(True)

# Create objects for animation
drone_body, = ax.plot([], [], 'bo-', lw=2)
thrust_L, = ax.plot([], [], 'r-', lw=1)
thrust_R, = ax.plot([], [], 'r-', lw=1)
path, = ax.plot([], [], 'k--', lw=1, alpha=0.5)

def update(frame):
    x, y, theta = Q_traj[0, frame], Q_traj[1, frame], Q_traj[2, frame]
    
    # Drone body
    drone_x = [x - L*casadi.cos(theta), x + L*casadi.cos(theta)]
    drone_y = [y - L*casadi.sin(theta), y + L*casadi.sin(theta)]
    drone_body.set_data(drone_x, drone_y) 
    
    # Thrust vectors
    if frame < N:
        thrust_scale = 0.1
        left_thrust = U_traj[0, frame] * thrust_scale
        right_thrust = U_traj[1, frame] * thrust_scale
        
        thrust_L.set_data([drone_x[0], drone_x[0]], 
                         [drone_y[0], drone_y[0] - left_thrust])
        thrust_R.set_data([drone_x[1], drone_x[1]], 
                         [drone_y[1], drone_y[1] - right_thrust])
    
    # Path
    path.set_data(Q_traj[0, :frame+1], Q_traj[1, :frame+1])
    
    return drone_body, thrust_L, thrust_R, path

anim = FuncAnimation(fig, update, frames=N+1, interval=dt*1000, blit=True)
plt.show()